In [1]:
import torch

In [5]:
from transformers import AutoTokenizer, OPTForCausalLM, BloomForCausalLM, CodeGenForCausalLM

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import transformers
transformers.__version__

'4.33.1'

In [7]:
model = OPTForCausalLM.from_pretrained("facebook/opt-125m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

In [6]:
# b_tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
# b_model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")

In [7]:
# c_tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
# c_model = CodeGenForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")

In [64]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), ep

In [75]:
def model_pre_hook(module, args, kwargs):
    # print(args, kwargs)
    display = {}
    for k, v in kwargs.items():
        if isinstance(v, torch.Tensor):
            display[k] = v.shape 
        elif k == 'past_key_values' and v is not None:
            display[k] = f'{len(v)} * ' + str((v[0][0].shape, v[0][1].shape))
        else:
            display[k] = v
    print(display)

    return args, kwargs

# model._forward_pre_hooks.clear()
# model.register_forward_pre_hook(model_pre_hook, with_kwargs=True)
# model._forward_pre_hooks

def layer_pre_hook(module, args, kwargs):
    display = {}
    for k, v in kwargs.items():
        if isinstance(v, torch.Tensor):
            display[k] = v.shape 
        elif k == 'past_key_value' and v is not None:
            display[k] = str((v[0].shape, v[1].shape))
        else:
            display[k] = v
    print(display)

    return args, kwargs

layer = model.model.decoder.layers[2]

layer._forward_pre_hooks.clear()
layer.register_forward_pre_hook(layer_pre_hook, with_kwargs=True)
layer._forward_pre_hooks


OrderedDict([(32, <function __main__.layer_pre_hook(module, args, kwargs)>)])

In [76]:
prompts = [
    'Who are you? Are you conscious?',
    'Where is Deutschland?',
    'How is Huawei Mate 60 Pro?'
] * 4

prompt_len = 20

inputs = tokenizer(prompts, padding="max_length", max_length=prompt_len, return_tensors="pt")

# Generate
generate_ids = model.generate(
    inputs.input_ids, 
    max_length=30 + prompt_len,
    # num_beams=2,
    # num_beam_groups=2,
    # diversity_penalty=0.1,
    # do_sample=True,
)

output_texts = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
for output_text in output_texts:
    print(output_text)
    print('-' * 10)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


{'attention_mask': torch.Size([12, 1, 20, 20]), 'layer_head_mask': None, 'past_key_value': None, 'output_attentions': False, 'use_cache': True}
{'attention_mask': torch.Size([12, 1, 1, 21]), 'layer_head_mask': None, 'past_key_value': '(torch.Size([12, 12, 20, 64]), torch.Size([12, 12, 20, 64]))', 'output_attentions': False, 'use_cache': True}
{'attention_mask': torch.Size([12, 1, 1, 22]), 'layer_head_mask': None, 'past_key_value': '(torch.Size([12, 12, 21, 64]), torch.Size([12, 12, 21, 64]))', 'output_attentions': False, 'use_cache': True}
{'attention_mask': torch.Size([12, 1, 1, 23]), 'layer_head_mask': None, 'past_key_value': '(torch.Size([12, 12, 22, 64]), torch.Size([12, 12, 22, 64]))', 'output_attentions': False, 'use_cache': True}
{'attention_mask': torch.Size([12, 1, 1, 24]), 'layer_head_mask': None, 'past_key_value': '(torch.Size([12, 12, 23, 64]), torch.Size([12, 12, 23, 64]))', 'output_attentions': False, 'use_cache': True}
{'attention_mask': torch.Size([12, 1, 1, 25]), 'laye